# Поиск с помощью регулярных выражений

Ещё документация по регулярным выражениям: https://habr.com/post/349860/

`\d` - цифра от 0 до 9;     
`\D` - что угодно, кроме цифр;     
`.` - в этом месте может находиться что угодно, кроме перевода строки `\n`;     
`+` - идущий перед ним символ может повторяться 1 или более раз;          
`*` - идущий перед ним символ может повторяться 0 или более раз;   
`?` - идущий перед ним символ может повторяться 0 или 1 раз.

`a\db = a0b, a1b, ..., a9b` <= между символами `a` и `b` обязана находиться одна цифра.       
`a\Db = azb, aab, a_b, a b, ...` <= между символами `a` и `b` должен находиться один символ и он обязан быть не цифрой.      
`a.b = a0b, a1b, aab, a b, a_b, ...` <= между символами `a` и `b` должен находиться любой символ, но только один.     
`a\d?b = ab, a5b, ...` <= между символами `a` и `b` должна находиться 1 цифра либо ничего.    
`a.*b = ab, a123XYZb, ...` <= между символами `a` и `b` может находиться что угодно в любом количестве.     


In [1]:
import re
html = 'Курс евро на сегодня 68,7514, курс евро на завтра 67,8901'
# Проверяется только первое вхождение (как только находим - записываем в переменную)
match = re.search('Евро\D+(\d+,\d+)', html)

In [2]:
re.search(r'Евро.*?(\d+,\d+)', html, re.IGNORECASE).group(1)
# Нашли первый курс

'68,7514'

In [3]:
re.findall(r'Евро\D+(\d+,\d+)', html, re.IGNORECASE)
# Нашли оба курса

['68,7514', '67,8901']

In [4]:
# Если группы не будет (запись без скобочек), то вывод будет всего, что попадёт под регулярные выражения
re.findall(r'Евро\D+\d+\d+', html, re.IGNORECASE)

['евро на сегодня 68', 'евро на завтра 67']

In [5]:
# Если групп будет несколько, то findall вернёт список кортежей
re.findall(r'Евро\D+(\d+),(\d+)', html, re.IGNORECASE)

# В каждом из кортежей будут значения всех групп указанных выражений

[('68', '7514'), ('67', '8901')]

In [6]:
# Во вложенных кортежах группы нумеруются по открывающимся скобкам
re.findall(r'Евро\D+((\d+),(\d+))', html, re.IGNORECASE)

[('68,7514', '68', '7514'), ('67,8901', '67', '8901')]

# Символьные классы

`[abcd1234] = [a-d1-4]` - один символ из множества;    
`\d = [0123456789] = [0-9]` - символьный класс, в который входят все цифры;    
`\D = [ˆ\d] = [ˆ0-9]` - символьный класс, в который **не** входят цифры;   

`a\d{2}c` - предыдущий символ должен повторяться ровно 2 раза;      
`a\d{,2}c` - предыдущий символ должен повторяться от 0 до 2 раз;      

`* = {0; inf}; + = {1; inf}; ? = {0; 1}`     

`\w = [a-zA-Z_0-9] + юникод` - буква, цифра и знак подчёркивания;     
`\W = [ˆ\w]` - может стоять что угодно, кроме букв, цифр и знаков подчёркивания;    
`\s` - пробел;    
`\S` - не пробел.


In [10]:
# Применяем метод re.compile, который позволяет один раз скомпилировать регулярное выражение,
# и потом несколько раз его использовать
# Удобно, чтобы несколько раз в большом коде или цикле не комплировать регулярное выражение заново.

### 1. Попробуем найти автомобильные номера в тексте

*Структура автомобильного номера*  

Номер: буква, 3 цифры, 2 буквы, 2-3 цифры;           
Буква не любая, а только [АВЕКМНОРСТУХ];     
`re.findall(pattern, where, re.IGNORECASE)`

In [7]:
import re
text = '''
Автомобиль с номером А123ВС77 подрезал автомобиль К654НЕ197, спровоцировав ДТП 
с участием еще двух иномарок с номерами М542ОР777 и О007ОО77
'''

# Сначала должны идти буквы из списка, потом ровно три цифры,
# затем снова буквы из списка, но они должны повторяться два раза.
# Затем должны быть две или три цифры. 
pattern = r'[АВЕКМНОРСТУХ]\d{3}[АВЕКМНОРСТУХ]{2}\d{2,3}'
re.findall(pattern, text)

['А123ВС77', 'К654НЕ197', 'М542ОР777', 'О007ОО77']

### 2. Поиск женских имён в тексте

In [13]:
# Применяем метод re.compile, который позволяет один раз скомпилировать регулярное выражение,
# и потом несколько раз его использовать
# Удобно, чтобы несколько раз в большом коде или цикле не комплировать регулярное выражение заново.

import re
text = (
    'Анна и Лена загорали на берегу океана, '
    'когда к ним подошли Яна и Ильнар'
)
re.findall(r'[А-Я]\w*на', text) # после каких-то букв в тексте ищем "на"

['Анна', 'Лена', 'Яна', 'Ильна']

Чтобы Ильна не было, добавим границы слова. Это будет означать, что ищем слово, которое *заканчивается* на "на".     
В начало и в конец регулярного выражения добавляем `\b`.

In [14]:
import re
text = (
    'Анна и Лена загорали на берегу океана, '
    'когда к ним подошли Яна и Ильнар'
)
re.findall(r'[А-Я]\w*на\b', text) # после каких-то букв в тексте ищем "на"

['Анна', 'Лена', 'Яна']

In [15]:
import re
text = (
    'Анна и Лена загорали на берегу океана, '
    'когда к ним подошли Яна, ПОЛИНА и Ильнар'
)
re.findall(r'[А-Я]\w*на\b', text)

# ПОЛИНА не нашлась, потому что мы ищем только те слова на маленькие буквы

['Анна', 'Лена', 'Яна']

Чтобы можно было найти имена, написанные и с маленьких букв, и с больших - напишем **группу**, в которой будут *на* и *НА*:

In [18]:
import re
text = (
    'Анна и Лена загорали на берегу океана, '
    'когда к ним подошли Яна, ПОЛИНА и Ильнар'
)
re.findall(r'[А-Я]\w*(на|НА)\b', text)

['на', 'на', 'на', 'НА']

Поставим `?:`. Это будет означать, что то, что находится в группе, группируется, но не запоминается.

In [20]:
import re
text = (
    'Анна и Лена загорали на берегу океана, '
    'когда к ним подошли Яна, ПОЛИНА и Ильнар'
)
re.findall(r'[А-Я]\w*(?:на|НА)\b', text) # после каких-то букв в тексте ищем "на"

['Анна', 'Лена', 'Яна', 'ПОЛИНА']

### 3.

In [28]:
import re
text = 'Как защитить металл от коррозии?'

# Заменяем все буквы "а" знаками вопроса
re.sub(r'а', '?', text)

'К?к з?щитить мет?лл от коррозии?'

In [33]:
# Найдём все большие буквы и выделим их в верхнем регистре
re.sub(r'(\w)\1', lambda r: r.group(0).upper(), text)

'Как защитить метаЛЛ от коРРозИИ?'